In [1]:
%matplotlib qt
from hcipy import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# magnification between first plane and considered plane
delta_gamma = 0.004

gamma_21 = 0.423
delta_gamma_21 = gamma_21 * delta_gamma

gamma_31 = 1.008
delta_gamma_31 =gamma_31 * delta_gamma

gamma_41 = 0.444
delta_gamma_41 = gamma_41 * delta_gamma

gamma_51 = 0.979
delta_gamma_51 = gamma_51 * delta_gamma

# P2 - Iris AO

In [3]:
p2_irisao_segment_size = 1.4e-3 # m
p2_irisao_segment_side_length = p2_irisao_segment_size / 2
p2_irisao_segment_gap_size = 12e-6 # m
p2_irisao_inscribed_circle_size = 10 * p2_irisao_segment_side_length
p2_irisao_flat_to_flat_size = 14 * np.sqrt(3) / 2 * p2_irisao_segment_side_length
p2_irisao_circumscribed_circle_size = np.sqrt(p2_irisao_flat_to_flat_size**2 + p2_irisao_segment_side_length**2)

In [4]:
irisao_gap_hexagon_fraction = p2_irisao_segment_gap_size / p2_irisao_segment_side_length
print(irisao_gap_hexagon_fraction, 0.012 / 0.7)

0.017142857142857144 0.017142857142857144


In [5]:
p2_irisao_distance_between_segments = p2_irisao_segment_side_length * np.sqrt(3)
p2_irisao_segment_circumdiameter = 2 * p2_irisao_segment_side_length - np.sqrt(3) * p2_irisao_segment_gap_size

def p2_irisao():
    segment = hexagonal_aperture(p2_irisao_segment_circumdiameter, np.pi/2)
    
    segment_positions = make_hexagonal_grid(p2_irisao_distance_between_segments, 3, False)
    
    aperture = make_segmented_aperture(segment, segment_positions)
    return aperture

# P1 - Pupil Mask

In [6]:
p1_pupil_mask_size = 19.850e-3 # m
p2_pupil_mask_size = p1_pupil_mask_size * gamma_21
print(p2_pupil_mask_size)

0.00839655


In [7]:
# Central segment
p1_pupil_mask_central_segment_size = 3.600e-3 # m
p2_pupil_mask_central_segment_size = p1_pupil_mask_central_segment_size * gamma_21
pupil_mask_central_segment_oversize_factor = p2_pupil_mask_central_segment_size / p2_irisao_segment_size
print(pupil_mask_central_segment_oversize_factor)

1.0877142857142856


In [8]:
# Spiders
p1_pupil_mask_spider_thickness = 0.200e-3 # m

In [9]:
pupil_mask_undersize_contour = p2_pupil_mask_size / p2_irisao_circumscribed_circle_size
print(pupil_mask_undersize_contour)

0.985988797756


In [10]:
# Output
print('Pupil mask side-to-side contour length in P1: %d um' % (p1_pupil_mask_size * 1e6))
print('Pupil mask central segment side length in P1: %d um' % (p1_pupil_mask_central_segment_size * 1e6))
print('Pupil mask spider thickness in P1: %d um' % (p1_pupil_mask_spider_thickness * 1e6))

Pupil mask side-to-side contour length in P1: 19850 um
Pupil mask central segment side length in P1: 3600 um
Pupil mask spider thickness in P1: 200 um


In [11]:
def p1_pupil_mask():
    segment = hexagonal_aperture(p1_pupil_mask_size / 7 / np.sqrt(3) * 2, np.pi / 2)
    distance_between_segments = p1_pupil_mask_size / 7
    segment_positions = make_hexagonal_grid(distance_between_segments, 3)
    contour = make_segmented_aperture(segment, segment_positions)
    
    central_segment = hexagonal_aperture(p1_pupil_mask_central_segment_size, np.pi / 2)
    
    spider1 = make_spider_infinite([0,0], 60, p1_pupil_mask_spider_thickness)
    spider2 = make_spider_infinite([0,0], 120, p1_pupil_mask_spider_thickness)
    spider3 = make_spider_infinite([0,0], -60, p1_pupil_mask_spider_thickness)
    spider4 = make_spider_infinite([0,0], -120, p1_pupil_mask_spider_thickness)
    
    def aper(grid):
        return (contour(grid) - central_segment(grid)) * spider1(grid) * spider2(grid) * spider3(grid) * spider4(grid)
    return aper

# P3 - Apodizer

In [12]:
# Contour
p3_apodizer_size = 19.725e-3 # m
p2_apodizer_size = p3_apodizer_size * gamma_21 / gamma_31
print(p2_apodizer_size)

0.008277455357142856


In [13]:
# Gap
p3_apodizer_mask_gap_size = 0.090e-3 # m
p3_irisao_segment_gap_size = p2_irisao_segment_gap_size * gamma_31 / gamma_21
apodizer_mask_gap_oversize_factor_wrt_irisao = p3_apodizer_mask_gap_size / p3_irisao_segment_gap_size

In [14]:
# Central segment
p3_apodizer_mask_central_segment_size = 3.950e-3 # m
p3_pupil_mask_central_segment_size = p1_pupil_mask_central_segment_size * gamma_31
apodizer_mask_central_segment_oversize_factor_wrt_pupil_mask = p3_apodizer_mask_central_segment_size / p3_pupil_mask_central_segment_size
p3_irisao_segment_size = p2_irisao_segment_size * gamma_31 / gamma_21

In [15]:
# Spiders
p3_apodizer_mask_spiders_thickness = 0.350e-3 # m

In [16]:
p3_irisao_segment_circumdiameter = p2_irisao_segment_circumdiameter * gamma_31 / gamma_21
p3_irisao_distance_between_segments = p2_irisao_distance_between_segments * gamma_31 / gamma_21
p3_apodizer_segment_circumdiameter = p3_irisao_segment_circumdiameter + (-p3_apodizer_mask_gap_size + p3_irisao_segment_gap_size) * np.sqrt(3)

def p3_apodizer():
    # segmentation
    segment = hexagonal_aperture(p3_apodizer_segment_circumdiameter, np.pi/2)
    segment_positions = make_hexagonal_grid(p3_irisao_distance_between_segments, 3, False)
    segmentation = make_segmented_aperture(segment, segment_positions)
    
    segment = hexagonal_aperture(p3_apodizer_size / 7 / np.sqrt(3) * 2, np.pi / 2)
    distance_between_segments = p3_apodizer_size / 7
    segment_positions = make_hexagonal_grid(distance_between_segments, 3)
    contour = make_segmented_aperture(segment, segment_positions)
    
    central_segment = hexagonal_aperture(p3_apodizer_mask_central_segment_size, np.pi / 2)
    
    spider1 = make_spider_infinite([0,0], 60, p3_apodizer_mask_spiders_thickness)
    spider2 = make_spider_infinite([0,0], 120, p3_apodizer_mask_spiders_thickness)
    spider3 = make_spider_infinite([0,0], -60, p3_apodizer_mask_spiders_thickness)
    spider4 = make_spider_infinite([0,0], -120, p3_apodizer_mask_spiders_thickness)
    
    def aper(grid):
        return segmentation(grid) * (contour(grid) - central_segment(grid)) * spider1(grid) * spider2(grid) * spider3(grid) * spider4(grid)
    return aper

# P4 - Boston DM

In [17]:
p4_boston_dm_size = 9.9e-3 # m
num_actuators = 34
actuator_pitch = p4_boston_dm_size / num_actuators

In [18]:
p4_apodizer_size = p3_apodizer_size * gamma_41 / gamma_31
p4_irisao_circumscribed_circle_size = p2_irisao_circumscribed_circle_size * gamma_41 / gamma_21
p4_irisao_flat_to_flat_size = p2_irisao_flat_to_flat_size * gamma_41 / gamma_21

In [19]:
boston_dm_undersize_contour_wrt_apodizer_mask = p4_apodizer_size / p4_boston_dm_size
boston_dm_contour_oversize_factor = p4_boston_dm_size / p4_irisao_circumscribed_circle_size

print(boston_dm_undersize_contour_wrt_apodizer_mask)
print(boston_dm_contour_oversize_factor)

0.87761544011544
1.10755089952


In [20]:
num_actuators_used = boston_dm_undersize_contour_wrt_apodizer_mask * num_actuators
print(num_actuators_used)

29.83892496392496


In [21]:
print('Actuator pitch: %d um' % (actuator_pitch * 1e6))
print('Number of used actuators: %0.2f' % (num_actuators_used))

Actuator pitch: 291 um
Number of used actuators: 29.84


In [22]:
def p4_boston_dm():
    return circular_aperture(p4_boston_dm_size)

# P5 - Lyot stop

In [23]:
p5_lyot_stop_size = 15.9e-3 # m
p5_irisao_inscribed_circle_size = p2_irisao_inscribed_circle_size * gamma_51 / gamma_21
lyot_stop_mask_undersize_contour_wrt_inscribed_circle = p5_lyot_stop_size / p5_irisao_inscribed_circle_size

p5_irisao_flat_to_flat_size = p2_irisao_flat_to_flat_size * gamma_51 / gamma_21
p5_irisao_circumscribed_circle_size = p2_irisao_circumscribed_circle_size * gamma_51 / gamma_21
print(p5_irisao_circumscribed_circle_size)

0.0197093010024


In [24]:
10 / (14 * np.sqrt(3) / 2) * lyot_stop_mask_undersize_contour_wrt_inscribed_circle

0.80946503210036624

In [25]:
# Central segment
p5_lyot_stop_mask_central_segment_size = 6.800e-3 # m
p5_apodizer_mask_central_segment_size = p3_apodizer_mask_central_segment_size * gamma_51 / gamma_31
print(p5_apodizer_mask_central_segment_size)
p5_irisao_segment_size = p2_irisao_segment_size * gamma_51 / gamma_21
lyot_stop_mask_central_segment_oversize_factor_wrt_apodizer_mask = p5_lyot_stop_mask_central_segment_size / p5_apodizer_mask_central_segment_size
print(lyot_stop_mask_central_segment_oversize_factor_wrt_apodizer_mask)

0.0038363591269841273
1.7725139318084842


In [26]:
# Spiders
p5_lyot_stop_mask_spiders_thickness = 0.700e-3 # m
lyot_stop_mask_spiders_thickness_ratio = p5_lyot_stop_mask_spiders_thickness / p5_irisao_circumscribed_circle_size
print(lyot_stop_mask_spiders_thickness_ratio)

0.0355162265731


In [27]:
def p5_lyot_stop():
    outer_diameter = circular_aperture(p5_lyot_stop_size)
    central_obscuration = circular_aperture(p5_lyot_stop_mask_central_segment_size)
    
    spider1 = make_spider_infinite([0,0], 60, p5_lyot_stop_mask_spiders_thickness)
    spider2 = make_spider_infinite([0,0], 120, p5_lyot_stop_mask_spiders_thickness)
    spider3 = make_spider_infinite([0,0], -60, p5_lyot_stop_mask_spiders_thickness)
    spider4 = make_spider_infinite([0,0], -120, p5_lyot_stop_mask_spiders_thickness)
    
    def aper(grid):
        return (outer_diameter(grid) - central_obscuration(grid)) * spider1(grid) * spider2(grid) * spider3(grid) * spider4(grid)
    return aper

In [28]:
g = make_pupil_grid(1024, p4_boston_dm_size / gamma_41 * gamma_21)
supersampling = 1

irisao = evaluate_supersampled(p2_irisao(), g, supersampling)
pupil_mask = evaluate_supersampled(p1_pupil_mask(), g.scaled(1 / gamma_21), supersampling)
apodizer = evaluate_supersampled(p3_apodizer(), g.scaled(gamma_31 / gamma_21), supersampling)
boston_dm = evaluate_supersampled(p4_boston_dm(), g.scaled(gamma_41 / gamma_21), supersampling)
lyot_stop = evaluate_supersampled(p5_lyot_stop(), g.scaled(gamma_51 / gamma_21), supersampling)

In [29]:
masks = [pupil_mask, irisao, apodizer, boston_dm, lyot_stop]
labels = ['Pupil Mask', 'IrisAO', 'Apodizer Mask', 'BostonDM', 'LyotStop']

for mask, label in zip(masks, labels):
    plt.figure()
    plt.title(label)
    imshow_field(mask, cmap='gray')
    plt.show()

In [30]:
for i, (mask_a, label_a) in enumerate(zip(masks, labels)):
    for j, (mask_b, label_b) in enumerate(zip(masks, labels)):
        if j <= i:
            continue
        
        plt.figure()
        plt.title(label_a + ' - ' + label_b)
        imshow_field(mask_a - mask_b, cmap='gray', vmin=-1, vmax=1)
        plt.show()

In [31]:
for gray in [True, False]:
    for num_pix in [2048]:
        grid = make_uniform_grid(num_pix, [p3_apodizer_size, p3_apodizer_size])

        own_apod = evaluate_supersampled(p3_apodizer(), grid, 1)
        write_fits(own_apod, 'ehpor_apodizer_mask_%d_%s.fits' % (num_pix, 'gy' if gray else 'bw'))

        own_lyot = evaluate_supersampled(p5_lyot_stop(), grid.scaled(gamma_51 / gamma_31), 4 if gray else 1)
        write_fits(own_lyot, 'ehpor_lyot_mask_%d_%s.fits' % (num_pix, 'gy' if gray else 'bw'))

KeyboardInterrupt: 

In [ ]:
imshow_field(own_apod - mathematica_apod)
plt.show()

In [ ]:
plt.subplot(1,2,1)
imshow_field(own_apod, cmap='gray')
plt.subplot(1,2,2)
imshow_field(own_lyot, cmap='gray')
plt.show()